# Short term Load Forcasting (Regression Machine Learning)

You have been provided with **Load Forecasting** data set as a single file, `dataset.csv`. 

We obtained it at http://www.mathworks.com/videos/electricity-load-and-price-forecasting-with-matlab-81765.html. 

For some background information you can also watch the video tutorial given in the link above:

Before you start on this notebook, copy the datasetset `dataset.csv` in the same directory.

### 1. Set up notebook and load dataset

In [ ]:
!pip install -r requirement.txt

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn import linear_model  # linear regression library
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


This next snippet of code loads the load forecasting dataset. There are 50000 data points, each with 8 predictor variables that are input `x` and one response variable (which we'll denote `y`).



In [ ]:
dataset = pd.read_csv('dataset.csv') # Reads the dataset in pandas dataframe

features = ['DryBulb', 'DewPoint', 'Hour', 'Weekday', 
            'IsWorkingDay', 'P_W_S_hour_l', 'P_D_S_hour_l', 'prev24HrAveLoad']
dataset.head() # This command tell the first 5 entries of the dataset

The dataset contains 8 input variable which contains the contextual information and historical load and temperature data. The variable includes:
- DryBulb temperature
- DewPoints
- Hours (data is collected in hourly samples so each day has 24 values)
- Weekday number [1 for Monday and 7 Sunday]
- Holyday flag [IsWorkingDay 0 for working 1 for holiday]
- prvious week same hour load (P_W_S_hour_l) 168 lagging entry i.e. 168 hours per week
- prevuous day same hour load (P_D_S_hour_l) 24 lagging entry
- prev24HrAveLoad (Average load for the perticular day)

Target variable is actual load at that perticular hour that is need to be predicted

# Statistical Feature Extraction (Increase the Feature Space)
This snippet of code extract the statistical features for the given data. These features includes the mean, standard deviation
and variance of daily, 48 hours, 72 hours and weekly load. 

In [ ]:

dataset['weekly_rollingavg'] = dataset['target_variable'].rolling(window=168).mean()
dataset['weekly_std'] = dataset['target_variable'].rolling(window=168).std()
dataset['daily_std'] = dataset['target_variable'].rolling(window=24).std()

dataset['weekly_varience'] = dataset['target_variable'].rolling(window=168).var()
dataset['daily_varience'] = dataset['target_variable'].rolling(window=24).var()


dataset['weekly_median'] = dataset['target_variable'].rolling(window=168).median()
dataset['daily_meadian'] = dataset['target_variable'].rolling(window=24).median()



dataset['48_hour_mean'] = dataset['target_variable'].rolling(window=48).mean()
dataset['48_hour_variance'] = dataset['target_variable'].rolling(window=48).var()
dataset['48_hour_std'] = dataset['target_variable'].rolling(window=48).std()


dataset['72_hour_mean'] = dataset['target_variable'].rolling(window=72).mean()
dataset['72_hour_variance'] = dataset['target_variable'].rolling(window=72).var()
dataset['72_hour_std'] = dataset['target_variable'].rolling(window=72).std()


dataset['12_hours_temp_mean'] = dataset['DewPoint'].rolling(window=12).mean()
dataset['24_hour_temp_mean'] = dataset['DewPoint'].rolling(window=24).mean()


dataset['12_hours_temp_std'] = dataset['DewPoint'].rolling(window=12).std()
dataset['24_hour_temp_std'] = dataset['DewPoint'].rolling(window=24).std()


dataset['12_hours_temp_var'] = dataset['DewPoint'].rolling(window=12).var()
dataset['24_hour_temp_var'] = dataset['DewPoint'].rolling(window=24).var()




In [ ]:
dataset.head() # Displays the first 5 values of expended dataset



In [ ]:
variables = [*dataset]

# The target_value is at index 8 so we have to remove it for the feature list.
#

index = 8
features = variables[:index]+variables[index+1:] # remove the target variable for feature list
print(features)

print('----------------------------------------------')

print('Number of features in training set:', len(features))



# Remove the missing value: (Data Filterig)

- While generating feature, the first few values of each column has NaN (not a number) which requires filtering before model training

In [ ]:
count=0
for i in dataset.isnull().sum(axis=1):
    if i>0:
        count=count+1
print('Total number of rows with missing values is ', count)
#print('Since it is only',round((count/len(dataset.index))*100), 'percent of the entire dataset the rows with missing values are excluded.')
dataset.isnull().sum()

### Some of the features have values that are need to be removed with NaN

In [ ]:
# Removeing the NaN and checking the dataset set. This is the inital filtering of the data.
dataset_new=dataset.dropna().reset_index(drop=True)
dataset_new

In [ ]:
count=0
for i in dataset_new.isnull().sum(axis=1):
    if i>0:
        count=count+1
print('Total number of rows with missing values is ', count)
#print('Since it is only',round((count/len(dataset.index))*100), 'percent of the entire dataset the rows with missing values are excluded.')
dataset_new.isnull().sum()

# Check the shape of the current dataset

In [ ]:
print('The shape of the dataset:', dataset_new.shape)

# out of 28, 27 are the input variables which includes the statisitcal and contextual features

# the target value is the "target_variable" which is predicted using machine learning

# The 'target_variable' is the actual hourly load on substation level.

# Preparing the Input and target values for model training

In [ ]:
# Seperating the input and target variables for and converting them into numpy array.



input_features = np.array(dataset_new.drop('target_variable', axis=1))         # Features
target_values = np.array(dataset_new['target_variable'])                       # Output


print('Shape of the Input data:', input_features.shape)
print('Shape of the target variable:', target_values.shape)


print('Data type of the input features:', input_features.dtype)

 ## One feature Regression Model


## Predict `y` using a single feature of `x`

Here we define a function, **one_feature_regression**, that takes `x` and `y`, along with the index `f` of a single feature and fits a linear regressor to `(x[f],y)`. It then plots the data along with the resulting line.

In [ ]:
def one_feature_regression(x,y,f):

    '''
    x -> one inpu feature

    y -> Target value

    f -> index of feature
    '''


    if (f < 0) or (f > 26):            # Because the total number of features in this dataset are 27
        print("Feature index is out of bounds")
        return
    regr = linear_model.LinearRegression()          # simple linear regressing model y=mx + c  where y output, x input, m slope, c intesecpt.
    x1 = x[:,[f]]
    regr.fit(x1, y)
    # Make predictions using the model
    y_pred = regr.predict(x1)
    # Plot data points as well as predictions
    plt.plot(x1, y, 'bo')
    plt.plot(x1, y_pred, 'r-', linewidth=3)
    plt.xlabel(features[f], fontsize=14)
    plt.ylabel('Load Progression', fontsize=14)
    plt.show()
    print("Mean squared error: ", mean_squared_error(y, y_pred))     # Loss function which will be reduced using machine learning
    print('Mean Absolute Percentage Errror', mean_absolute_percentage_error(y, y_pred)*100)
    return regr

### (a) You have been given one_feature_regression function. Please change the index f to change the feature and develop a regression model. Compute the MSE, w, and b? (b).

In [ ]:
# In the given example we are using the feature number 7 to train the regression model

OneF_reg = one_feature_regression(input_features,target_values,7)    # Fearure is prev24HrAvgLoad


w = OneF_reg.coef_
b = OneF_reg.intercept_

# Weights of regress

print('Weights is %f, and Bais is %f', (w, b))    # 1 feature: 1 weight

# This function provides the single feature regress along with the value of loss and MAPE. 
# The x-axis give the name of feature used for regression.


# Find the one feature with mininmim MSE or MAPE. (Hint: simply substitute the f index in the above function calling.

### (b) Identify the second-best feature with the lowest MSE.

## Subset feature Regression Model

##  Predict `y` using a specified subset of features from `x`

The function **feature_subset_regression** is just like **one_feature_regression**, but this time uses a list of features `flist`.

In [ ]:
def feature_subset_regression(x,y,flist):

    '''
    x -> input features
    y -> target values

    flist -> subset of feature for model training: pass list of the feature intex  [6,7] for feature number 7 and 8
    '''


    if len(flist) < 1:
        print("Need at least one feature")
        return
    for f in flist:
        if (f < 0) or (f > 26):
            print("Feature index is out of bounds")
            return
    regr = linear_model.LinearRegression()
    x1 = x[:,flist]
    regr.fit(x1, y)
    # Make predictions using the model
    y_pred = regr.predict(x1)

    print("Mean squared error: ", mean_squared_error(y, y_pred))
    print('Mean Absolute Percentage Errror', mean_absolute_percentage_error(y, y_pred)*100)
    return regr

### (a) Using the *feature_subset_regression*, find out the model performance in terms of MSE using features #7 (P_D_S_hour_l) and #8 (prev24HrAveLoad)?

In [ ]:
# In this example we will use the feature number 6,7,8,9 and for regressing


mul_f = feature_subset_regression(input_features, target_values, [5,6,7,8])   # rememeber index starts for 0

# With multiple feature feature the loss function starts to decrease. 


w = mul_f.coef_
b = mul_f.intercept_

# Weights of regress

print('Weights is %f, and Bais is %f', (w, b))   # four feature four weights


### (b) Use all 27 Features for regression analysis and compare the model performance?

In [ ]:
# Using all features for regression

feature_list = [i for i in range(27)]

reg = feature_subset_regression(input_features, target_values, feature_list)


w = reg.coef_
b = reg.intercept_

# Weights of regress

print('Length Weights is %f, and Bais is %f', (len(w), b))   # 27 feature four weights

print('---------------------------------------------')

print('Length Weights is %f, and Bais is %f', (w, b))


## Finding the best Train-Test Data Split


## Splitting the data into a training and test set

In the experiments above, every model was fit to the *entire* data set and its mean squared error was evaluated on this same data set. This methodology would not, in general, yield accurate estimates of future error. In this specific case, however, the discrepancy might not be too bad because the data set is quite large relative to the number of features.

To investigate this further, we define a procedure **split_data** that partitions the data set into separate training and test sets. It is invoked as follows:

* `trainx, trainy, testx, testy = split_data(n_train)`

Here:
* `n_train` is the desired number of training points
* `trainx` and `trainy` are the training points and response values
* `testx` and `testy` are the test points and response values

The split is done randomly, but the random seed is fixed, and thus the same split is produced if the procedure is called repeatedly with the same `n_train` parameter.

In [ ]:
def split_data(n_train):
    if (n_train < 0) or (n_train > 48000):
        print("Invalid number of training points")
        return
    np.random.seed(0)
    perm = np.random.permutation(48000)
    training_indices = perm[range(0,n_train)]
    test_indices = perm[range(n_train,48000)]
    trainx = input_features[training_indices,:]
    trainy = target_values[training_indices]
    testx = input_features[test_indices,:]
    testy = target_values[test_indices]
    return trainx, trainy, testx, testy

### (a)  Using the **split_data** procedure to partition the data set, compute the training MSE and test MSE when fitting a regressor to *all* features, for the following training set sizes:
* `n_train = 20000`
* `n_train = 25000`
* `n_train = 30000`
* `n_train = 40000`

In [ ]:
n_train = 40000

# splitting the data
xtrain, ytrain, xtest, ytest =split_data(n_train)

# Generating the model
reg = linear_model.LinearRegression()

# Model training
reg.fit(xtrain, ytrain)

# Prediction

y_pred = reg.predict(xtest)

# MSE
mse = mean_squared_error(ytest, y_pred)
MAPE = mean_absolute_percentage_error(ytest, y_pred)*100

print('For Training size=%d, the test MSE is :%f, and MAPE is :%f', n_train, mse, MAPE)


# You can change the values of n_train and see the impact on MSE and MAPE

### (b) What is the impact of increasing the training size on the performance of the model?


## Data Normalisation

###  (a) Apply Min-Max feature scaling to normalize the data and caluclate the MSE and MAPE on features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_scaled=scaler.fit_transform(input_features)



# training the model using scaled features

reg_scaled = feature_subset_regression(x_scaled, target_values, feature_list) 

# function defined previously for using subset and all features for regression.



## Comparing Decision-Tree Versus Random Forest Regression Algorithms

### (a) Use test train split to divide the dataset in 70-30 and applying the decision tree and Random forest algorithms using sklearn library

Find the MSE and MAPE for both decision tree and random forest and compare it with linear regression and Plot the actual and predicted results of first 168 hours (weekly plot)

In [ ]:
# Importing the libraries for decision tree and random forest for sklearn

from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor
import time

# Decision Tree Using sklearn

In [ ]:
# Creating the test train split using sklearn library

# split is kept 70 - 30

# the data is not shaffled

x_train, x_test, y_train, y_test = train_test_split(input_features,target_values, test_size=0.3, 
                                                        random_state=42, shuffle=False)  

In [ ]:
'''
DecisionTreeRegressor(*, criterion='squared_error', splitter='best', max_depth=None, min_samples_split=2, 
                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                        max_features=None, random_state=None, max_leaf_nodes=None, 
                        min_impurity_decrease=0.0, ccp_alpha=0.0)

'''

# creating a DF using the default parameters

DT = DecisionTreeRegressor()    # Change parameter, write the parameter name and values in the function called

# Model training

start = time.time()
DT = DT.fit(x_train, y_train)
stop = time.time()

dt_time = stop-start

# model prediction

y_pred_dt = DT.predict(x_test)

# model evaluation

print('Model Performance')

print('-----------------------------------------------------------')

print('Decision Tree convergence time in seconds:', dt_time)

print('-----------------------------------------------------------')

print('MSE:', mean_squared_error(y_test, y_pred_dt))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred_dt)*100)



To further optimise the model and improve performance, the parameters of the model are change.

The process of obtaining the optimal model parameter is termed as hyperparameter tuning.

# Weekly Plot of Decision Tree 

In [ ]:
# Plot the the results of actual vs predicted values

plt.plot(y_test[1800:1968], 'b--', label='Actual')
plt.plot(y_pred_dt[1800:1968], 'r-', label='Predicted')
plt.title('Decision Tree Results')
plt.xlabel('Hourly Time')
plt.ylabel('Load in MW')

plt.legend(loc='upper left')

figsize=(20,16)

# Random Forest Using Sklearn

In [ ]:
''' 
RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, 
                    min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, 
                    min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, 
                    verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
'''


# Parameters for random forest classifier
# In this case, instead of using the default parameter, some of the parameter are been changed
# Further, the parameter tuning is done using either grid search, random search or other techniques


from random import random
from tkinter import N
from sklearn.ensemble import RandomForestRegressor


# create the model

rf = RandomForestRegressor(n_estimators=200, max_depth=30, n_jobs=-1, random_state=42)

# train the model

start = time.time()

rf.fit(x_train, y_train)

stop = time.time()

rf_time = stop-start
# model predction

y_pred_rf = rf.predict(x_test)



# model evaluation

print('Model Performance')

print('-----------------------------------------------------------')

print('Random Forest Convergence time in seconds:', rf_time)

print('-----------------------------------------------------------')

print('MSE:', mean_squared_error(y_test, y_pred_rf))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred_rf)*100)


In [ ]:
# Plot the the results of actual vs predicted values

plt.plot(y_test[1800:1968], 'b--', label='Actual')
plt.plot(y_pred_rf[1800:1968], 'r-', label='Predicted')
plt.title('Random Forest Results')
plt.xlabel('Hourly Time')
plt.ylabel('Load in MW')

plt.legend(loc='upper left')

figsize=(20,16)

For the above results, it is clearly evident that Random Forest performs better the DT and Linear Regression of this particular problem. 

The original dataset is expended to obtain 27 feature vecotors and analysis is done. Using linear regression, a short investigation is done on 
the impact of different feature set on the performace of model

Decision Tress and Random Forest models are trained and evaluated using the 27 feature and results shows the performace of RF is better then DT.


However, in machine learning the input feature set has a huge impact on the performance of the model. Therefore, we will perform an analysis on the
feature importance and overall impact on the model performmance. Further, will smaller feature set the model convergence can be reduced.

In [ ]:
import session_info
session_info.show()

In [ ]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,watermark,sklearn,matplotlib,keras

# date
print (" ")
%watermark -u -n -t -z 